### Sample code - forks count

In [1]:
import pandas as pd
merge = pd.read_csv('data/merge_bq_repo_name.csv') # merge_bq_repo_name is the repo name I merged 
merge.shape

(16467, 2)

In [2]:
merge.head()

,repo_name,company_name
0,000webhost/000webhost-sdk-js,000webhost
1,100dayproject/polymer-web-example,100dayproject
2,18F/18f-cf-release,18F
3,18F/api-program,18F
4,18F/caddy-boshrelease,18F


In [3]:
merge_repo = list(merge['repo_name']) # get only repo name from dataset
len(merge_repo)

16467

### Create pickle to save data in every iteration

In [55]:
# -- save picle  
import pickle
filepath = 'fork_time.pkl'

def batch(time_list):
    
    with open(filepath, 'ab') as f:
        pickle.dump(time_list, f)

In [ ]:
import requests
from itertools import chain
auth = ('vivi-yu05', 'c968bef9cda4907d6280ba241a72043360584715') # this can be get from your github account
fork_time = []

for j in range(len(merge_repo)):

    page = 1
    while True: 
        
        # use github API to scrape data
        get = requests.get('https://api.github.com/repos/'+ str(merge_repo[j]) +'/forks?page='+ str(page) +'&per_page=100',
                      auth = auth)
        get_json = json.loads(get.text)
        
        if len(get_json) == 0: # if there is nothing inside, break out of the loop and look for next repo
            break
        for i in range(len(get_json)):
            repo_query_list = ['full_name', 'created_at', 'pushed_at', 'updated_at'] # get only this data
            
            repo_tuple = tuple([merge_repo[j]]) # save the repo name first
            repo_tuple += tuple(get_json[i][x] for x in repo_query_list) # save data in repo_quiery_list
            batch(repo_tuple) # call batch function to save repo_tuple in a pickle file
        page += 1 
    print(j) # print which repo we are scraping
    
    

### Read from pickle

In [90]:
colnames = ['repo_name', 'fork_full_name', 'created_at', 'pushed_at', 'updated_at'] # set column names

info = []
with open('fork_time.pkl', 'rb') as fr:
    try:
        while True:
            info.append(pickle.load(fr))
    except EOFError:
        pass
    
repo_info = pd.DataFrame(info, columns=colnames)
repo_info

,repo_name,fork_full_name,created_at,pushed_at,updated_at
0,000webhost/000webhost-sdk-js,yuutaasaba/000webhost-sdk-js,2018-05-06T20:08:50Z,2018-04-13T11:52:01Z,2018-05-06T20:08:52Z
1,000webhost/000webhost-sdk-js,minotoki/000webhost-sdk-js,2017-10-30T08:04:02Z,2017-10-25T07:49:06Z,2017-10-30T08:04:04Z
2,18F/18f-cf-release,CNXTEoEorg/cg-cf-release,2017-07-14T04:24:44Z,2017-06-01T00:01:36Z,2017-07-14T04:24:46Z
3,18F/18f-cf-release,cnelson/cg-cf-release,2016-04-19T14:03:21Z,2016-05-05T14:34:56Z,2016-04-19T14:03:22Z
4,18F/api-program,CNXTEoEorg/api-program,2017-07-14T05:12:50Z,2017-04-27T17:09:29Z,2017-07-14T05:12:52Z
5,18F/api-program,jrtorres42/api-program,2016-08-28T05:37:47Z,2016-08-21T23:53:30Z,2016-08-28T05:37:48Z
6,18F/api-program,din982/api-program,2016-08-12T11:01:12Z,2016-08-10T16:07:00Z,2016-08-12T11:01:18Z
7,18F/api-program,djeraseit/api-program,2016-06-07T23:53:03Z,2016-04-28T18:11:06Z,2016-06-07T23:53:04Z
8,18F/api-program,GUI/api-program,2016-03-02T20:02:55Z,2016-03-02T20:21:03Z,2016-03-02T20:02:55Z
9,18F/api-program,OpenGlobe/api-program,2016-01-29T20:44:41Z,2016-01-29T20:56:23Z,2016-01-29T20:44:44Z
